In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchvision import transforms, datasets
import torch.backends.cudnn as cudnn

import time
import numpy as np 
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import math
from scipy.special import softmax

from networks import MCDropout
from utils import *
from utils_plotting import *

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
use_cuda = torch.cuda.is_available()

## Dataset

In [ ]:
batch_size = 128
mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.2010)

transform_train = transforms.Compose([transforms.RandomHorizontalFlip(),
                                    transforms.RandomCrop(32, padding=4),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean, std)
                                    ])
transform_test = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize(mean, std)])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
valset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=4)

models_dir = "Models/" + 'MCdrop_models'
results_dir = "Results/" + 'MCdrop_results'

# savemodel_its = [10,20,30,40,50]
# save_dicts = []


## Network

In [ ]:
# net = MC_drop_net(lr=lr, channels_in=3, side_in=32, cuda=use_cuda, classes=10, batch_size=batch_size, weight_decay=weight_decay)
net = MCDropout(18, 10)

## Training

In [ ]:
if use_cuda:
    net.cuda()
    net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True

In [ ]:
criterion = nn.CrossEntropyLoss()

def train(epoch): 
    net.train()
    train_loss = 0
    correct = 0
    total = 0

    lr = 0.1
    
    optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate(lr, epoch), momentum=0.9, weight_decay=5e-4)

    print('\n=> Training Epoch #%d, LR=%.4f' %(epoch, learning_rate(lr, epoch)))
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda() # GPU settings
        optimizer.zero_grad()
        inputs, targets = Variable(inputs), Variable(targets)
        outputs = net(inputs)               # Forward Propagation
        loss = criterion(outputs, targets)  # Loss
        loss.backward()  # Backward Propagation
        optimizer.step() # Optimizer update

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

        # sys.stdout.write('\r')
        # sys.stdout.write('| Epoch [%3d/%3d] Iter[%3d/%3d]\t\tLoss: %.4f Acc@1: %.3f%%'
        #         %(epoch, num_epochs, batch_idx+1,
        #             (len(trainset)//batch_size)+1, loss.item(), 100.*correct/total))
        # sys.stdout.flush()
    print('| Epoch [%3d/%3d] Iter[%3d/%3d]\t\tLoss: %.4f Acc@1: %.3f%%'
                %(epoch, num_epochs, batch_idx+1,
                    (len(trainset)//batch_size)+1, loss.item(), 100.*correct/total))

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(valloader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        with torch.no_grad():
            inputs, targets = Variable(inputs), Variable(targets)
        outputs = net(inputs)
        loss = criterion(outputs, targets)

        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    # Save checkpoint when best model
    acc = 100.*correct/total
    print("\n| Validation Epoch #%d\t\t\tLoss: %.4f Acc@1: %.2f%%" %(epoch, loss.item(), acc))

    if acc > best_acc:
        print('| Saving Best model...\t\t\tTop1 = %.2f%%' %(acc))
        state = {
                'net':net.module if use_cuda else net,
                'acc':acc,
                'epoch':epoch,
        }

        torch.save(state, models_dir + '/' + 'theta_best.t7')
        best_acc = acc

def get_hms(seconds):
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)

    return h, m, s

In [ ]:
num_epochs = 200
best_acc = 0

start = time.time()
elapsed_time = 0
for epoch in range(num_epochs):
    start_time = time.time()

    train(epoch)
    test(epoch)

    epoch_time = time.time() - start_time
    elapsed_time += epoch_time
    print('| Elapsed time : %d:%02d:%02d'  %(get_hms(elapsed_time)))

print('\n[Phase 4] : Testing model')
print('* Test results : Acc@1 = %.2f%%' %(best_acc))
end = time.time()
print('| Total time : %d:%02d:%02d'  %(get_hms(end - start)))

In [ ]:
#91.30
#89.51

## Inference

In [ ]:
checkpoint = torch.load(models_dir + '/' + 'theta_best.t7')
net = checkpoint['net']

###  CIFAR-10 Rotations

In [ ]:
# rotations, Bayesian
# valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=4)
x_dev = []
y_dev = []
for x, y in valloader:
    x_dev.append(x.cpu().numpy())
    y_dev.append(y.cpu().numpy())

x_dev = np.concatenate(x_dev)
y_dev = np.concatenate(y_dev)
print(x_dev.shape)
print(y_dev.shape)

In [ ]:
plot_rotate(x_dev, y_dev, net, results_dir, im_list = valset.test_data ,im_ind = 23, Nsamples = 100, steps=16)

In [ ]:
data_rotated = np.load("data/CIFAR10_rotated.npy")
# data_rotated = np.transpose(data_rotated, (0,1,4,2,3))
data_rotated.shape

In [ ]:
steps = 16
N = 10000
Nsamples = 100
y_dev = valset.test_labels

def preprocess_test(X):

    N, H, W, C = X.shape
    Y = torch.zeros(N, C, H, W)
    mean = (0.4914, 0.4822, 0.4465)
    std = (0.2023, 0.1994, 0.2010)  
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean,std),
    ])

    for n in range(len(X)):
        Y[n] =  transform_test(X[n])

    return Y

all_preds = np.zeros((N, steps, 10))
all_sample_preds = np.zeros((N, Nsamples, steps, 10))

net.eval()
for im_ind in range(N):
    if(im_ind % 500 == 0):
        print(im_ind)

    y =  y_dev[im_ind]
    
    ims = data_rotated[:,im_ind,:,:,:]
    ims = preprocess_test(ims)

    y = np.ones(ims.shape[0])*y
    
    with torch.no_grad():
        sample_probs = net.all_sample_eval(ims, torch.from_numpy(y), Nsamples=Nsamples)
    probs = sample_probs.mean(dim=0)
    
    all_sample_preds[im_ind, :, :, :] = sample_probs.cpu().numpy()
    predictions = probs.cpu().numpy()
    all_preds[im_ind, :, :] = predictions

In [ ]:
rotations = (np.linspace(0, 179, steps)).astype(int)

correct_preds = np.zeros((N, steps))
for i in range(N):
    correct_preds[i,:] = all_preds[i,:,y_dev[i]]   

np.save(results_dir+'/correct_preds.npy', correct_preds)
np.save(results_dir+'/all_preds.npy', all_preds)
np.save(results_dir+'/all_sample_preds.npy', all_sample_preds)

plot_predictive_entropy(correct_preds, all_preds, rotations, results_dir)

### CIFAR-10-C

In [ ]:
chalPath = 'data/CIFAR-10-C/'
chals = sorted(os.listdir(chalPath))

chal_labels = valset.test_labels
chal_labels = torch.Tensor(chal_labels)
chal_labels = chal_labels.long()

def preprocess_test(X):

    N, H, W, C = X.shape
    Y = torch.zeros(N, C, H, W)
    mean = (0.4914, 0.4822, 0.4465)
    std = (0.2023, 0.1994, 0.2010)  
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean,std),
    ])

    for n in range(len(X)):
        Y[n] =  transform_test(X[n])

    return Y

preds_list= []
net.eval()
avg_list = []

for challenge in range(len(chals)):
    chal_data = np.load(chalPath + chals[challenge])
    # chal_data = np.transpose(chal_data, (0,3,1,2))

    avg = 0
    for j in range(5):
        chal_temp_data = chal_data[j * 10000:(j + 1) * 10000]
        chal_temp_data = preprocess_test(chal_temp_data)

        chal_dataset = torch.utils.data.TensorDataset(chal_temp_data, chal_labels)
        chal_loader = torch.utils.data.DataLoader(chal_dataset, batch_size=100)
        chal_error = 0

        with torch.no_grad():
            for x, y in chal_loader:
                cost, err, probs = net.sample_eval(x, y, Nsamples=10, logits=False)
                preds_list.append(probs)
                chal_error += err.cpu().numpy()
                # print(err)

        # print(chal_error)
        chal_acc = 1 - (chal_error/len(chal_dataset))
        avg += chal_acc
        print(chal_acc)
    
    avg /= 5
    avg_list.append(avg)
    print("Average:",avg," ", chals[challenge])

print("Mean: ", np.mean(avg_list))

In [ ]:
preds_list = np.vstack(preds_list)
np.save(results_dir+'/preds_CIFAR-10-C.npy', preds_list)
np.save(results_dir+'/avg_list_CIFAR-10-C.npy', avg_list)